In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import mplhep as hep
hep.style.use("CMS")
from coffea import util
import itertools
import os, sys
import glob
import copy
import uproot
import json

import mplhep as hep
hep.style.use("CMS")

sys.path.append('../python/')
import functions

In [2]:
IOV = '2018'

In [3]:
saved = '../outputs/scale'


qcd16 = util.load(f'{saved}/QCD_2016all.coffea')
qcd17 = util.load(f'{saved}/QCD_2017.coffea')
qcd18 = util.load(f'{saved}/QCD_2018.coffea')


ttbar17 = util.load(f'{saved}/TTbar_2017.coffea')
ttbar16 = util.load(f'{saved}/TTbar_2016all.coffea')
ttbar18 = util.load(f'{saved}/TTbar_2018.coffea')


data17 = util.load(f'{saved}/JetHT_2017.coffea')
data16 = util.load(f'{saved}/JetHT_2016all.coffea')
data18 = util.load(f'{saved}/JetHT_2018.coffea')


data16_apv = util.load(f'{saved}/JetHT_2016APV.coffea')
data16_noapv = util.load(f'{saved}/JetHT_2016.coffea')

data16_cutflow = {}
data16_cutflow['cutflow'] = {}

for key in data16_apv['cutflow'].keys():
    data16_cutflow['cutflow'][key] = data16_apv['cutflow'][key] + data16_noapv['cutflow'][key]

# qcd17 = util.load(f'../outputs/scale/QCD_2017.coffea')
# ttbar17 = util.load(f'../outputs/scale/TTbar_2017.coffea')
# data17 = util.load(f'../outputs/scale/JetHT_2017.coffea')

# qcd16 = util.load(f'../outputs/scale/QCD_2016all.coffea')
# ttbar16 = util.load(f'../outputs/scale/TTbar_2016all.coffea')
# data16 = util.load(f'../outputs/scale/JetHT_2016all.coffea')



## Cutflow

In [4]:
data = {}
cuts = []

for cut in data16['cutflow'].keys():
    if 'sumw' in cut: continue
        
    evts = data18['cutflow'][cut] + data17['cutflow'][cut] + data16_cutflow['cutflow'][cut]
    data[cut] = evts
    cuts.append(evts)

        

In [5]:
df = pd.DataFrame(data=cuts, index=data.keys())
df

,0
all events 1,835683606
all events,713887608
trigger,52300242
htCut,50703368
metfilter,50469604
jetid,50468108
jetkincut,46928212
twoFatJets,34825602
oneTTbar,34825602
dPhiCut,34663746


In [6]:
table1 = '''
\\begin{table}
    \\begin{center}
        \\rowcolors{2}{gray!15}{white}
        \\begin{tabular}{ |p{6cm}||p{3cm}||p{3cm}||p{3cm}||p{3cm}|  } \\hline 
            \\textbf{Cut} & \\textbf{2016 Events} & \\textbf{2017 Events} & \\textbf{2018 Events} & \\textbf{Total Events} \\\\ \\hline
            All Events                                  &  '''+'{0:0,.0f}'.format(data16_cutflow['cutflow']['all events'])+'''  &  '''+'{0:0,.0f}'.format(data17['cutflow']['all events'])+''' &  '''+'{0:0,.0f}'.format(data18['cutflow']['all events'])+'''  &  '''+'{0:0,.0f}'.format(data['all events'])+''' \\\\
            Trigger                                     &  '''+'{0:0,.0f}'.format(data16_cutflow['cutflow']['trigger'])+'''  &  '''+'{0:0,.0f}'.format(data17['cutflow']['trigger'])+''' &  '''+'{0:0,.0f}'.format(data18['cutflow']['trigger'])+'''  &  '''+'{0:0,.0f}'.format(data['trigger'])+''' \\\\
            $HT > 1400$                                 &  '''+'{0:0,.0f}'.format(data16_cutflow['cutflow']['htCut'])+'''  &  '''+'{0:0,.0f}'.format(data17['cutflow']['htCut'])+''' &  '''+'{0:0,.0f}'.format(data18['cutflow']['htCut'])+'''  &  '''+'{0:0,.0f}'.format(data['htCut'])+''' \\\\
            METFilter                                   &  '''+'{0:0,.0f}'.format(data16_cutflow['cutflow']['metfilter'])+'''  &  '''+'{0:0,.0f}'.format(data17['cutflow']['metfilter'])+''' &  '''+'{0:0,.0f}'.format(data18['cutflow']['metfilter'])+'''  &  '''+'{0:0,.0f}'.format(data['metfilter'])+''' \\\\
            $\\ge 1$ AK8 jet passing JetID               &  '''+'{0:0,.0f}'.format(data16_cutflow['cutflow']['jetid'])+'''  &  '''+'{0:0,.0f}'.format(data17['cutflow']['jetid'])+''' &  '''+'{0:0,.0f}'.format(data18['cutflow']['jetid'])+'''  &  '''+'{0:0,.0f}'.format(data['jetid'])+''' \\\\
            $\\ge 1$ AK8 $p_T > 400$ GeV, $|y| < 2.4$    &  '''+'{0:0,.0f}'.format(data16_cutflow['cutflow']['jetkincut'])+'''  &  '''+'{0:0,.0f}'.format(data17['cutflow']['jetkincut'])+''' &  '''+'{0:0,.0f}'.format(data18['cutflow']['jetkincut'])+'''  &  '''+'{0:0,.0f}'.format(data['jetkincut'])+''' \\\\
            $>2$ AK8 Jets                               &  '''+'{0:0,.0f}'.format(data16_cutflow['cutflow']['twoFatJets'])+'''  &  '''+'{0:0,.0f}'.format(data17['cutflow']['twoFatJets'])+''' &  '''+'{0:0,.0f}'.format(data18['cutflow']['twoFatJets'])+'''  &  '''+'{0:0,.0f}'.format(data['twoFatJets'])+''' \\\\
            $|\\Delta\\varphi| > 2.1$                     &  '''+'{0:0,.0f}'.format(data16_cutflow['cutflow']['dPhiCut'])+'''  &  '''+'{0:0,.0f}'.format(data17['cutflow']['dPhiCut'])+''' &  '''+'{0:0,.0f}'.format(data18['cutflow']['dPhiCut'])+'''  &  '''+'{0:0,.0f}'.format(data['dPhiCut'])+''' \\\\
            \\hline
        \\end{tabular}
        \\caption{Cutflow for blinded JetHT events for 2016, 2017 and 2018 datasets.}
        \\label{tab:cutflow}
    \\end{center}
\\end{table}

'''

In [7]:
print(table1)
print(table1, file=open('view.txt', 'w'))


\begin{table}
    \begin{center}
        \rowcolors{2}{gray!15}{white}
        \begin{tabular}{ |p{6cm}||p{3cm}||p{3cm}||p{3cm}||p{3cm}|  } \hline 
            \textbf{Cut} & \textbf{2016 Events} & \textbf{2017 Events} & \textbf{2018 Events} & \textbf{Total Events} \\ \hline
            All Events                                  &  263,553,842  &  173,256,968 &  277,076,798  &  713,887,608 \\
            Trigger                                     &  24,189,590  &  13,650,082 &  14,460,570  &  52,300,242 \\
            $HT > 1400$                                 &  22,781,430  &  13,520,202 &  14,401,736  &  50,703,368 \\
            METFilter                                   &  22,689,182  &  13,453,484 &  14,326,938  &  50,469,604 \\
            $\ge 1$ AK8 jet passing JetID               &  22,688,074  &  13,453,320 &  14,326,714  &  50,468,108 \\
            $\ge 1$ AK8 $p_T > 400$ GeV, $|y| < 2.4$    &  19,709,796  &  13,154,242 &  14,064,174  &  46,928,212 \\
            $>2$ 

## Weights

In [9]:
wgts = {}
wgts['2018'] = ttbar18['weights']
wgts['2017'] = ttbar17['weights']
wgts['2016'] = ttbar16['weights']

wdata = {}
wdata2 = {}

for year in ['2016', '2017', '2018']:
    
    wdata[year] = {}
    wdata2[year] = {}
    nom = wgts[year]['nominal']
    
    for wgt in wgts[year]:
        
#         print(wgt)
        
        
        if not 'nominal' in wgt:
            
            try:

                wdata2[year][wgt] = wgts[year][wgt]/nom

                wdata[year][wgt] = np.abs(1-wgts[year][wgt]/nom) #np.abs(1-np.abs(wgts[wgt]/nom))
            except:
                continue


In [10]:
for year in wdata2.keys():
    print(year)
    
    for key in wdata2[year].keys():
    
        print(key, '{0:0.3f}'.format(wdata2[year][key]), '{0:0.3f}'.format(wdata[year][key]))

2016
prefiringDown 1.007 0.007
q2Up 1.377 0.377
btagDown 0.935 0.065
pileupUp 1.003 0.003
pdfUp 1.007 0.007
btagUp 1.067 0.067
q2Down 0.727 0.273
pileupDown 0.997 0.003
pdfDown 1.014 0.014
prefiringUp 0.992 0.008
jesUp 1.056 0.056
jesDown 0.945 0.055
jerUp 1.005 0.005
jerDown 0.994 0.006
2017
q2Down 0.728 0.272
q2Up 1.372 0.372
pdfUp 1.007 0.007
pileupDown 0.997 0.003
pileupUp 1.003 0.003
btagUp 1.081 0.081
btagDown 0.922 0.078
prefiringUp 0.993 0.007
pdfDown 1.014 0.014
prefiringDown 1.007 0.007
jesUp 1.047 0.047
jesDown 0.955 0.045
jerUp 1.009 0.009
jerDown 0.992 0.008
2018
q2Down 0.727 0.273
btagUp 1.078 0.078
pileupDown 0.999 0.001
q2Up 1.375 0.375
btagDown 0.924 0.076
pileupUp 1.001 0.001
pdfDown 1.014 0.014
pdfUp 1.007 0.007
jesUp 1.057 0.057
jesDown 0.948 0.052
jerUp 1.006 0.006
jerDown 0.994 0.006


In [11]:
wdata2

{'2016': {'prefiringDown': 1.0074647626532929,
  'q2Up': 1.3767351703904642,
  'btagDown': 0.9345142942108225,
  'pileupUp': 1.0027166713114262,
  'pdfUp': 1.007107769672137,
  'btagUp': 1.0672233280206287,
  'q2Down': 0.7274360476460707,
  'pileupDown': 0.9973733302199356,
  'pdfDown': 1.0142155393442744,
  'prefiringUp': 0.9923427240670517,
  'jesUp': 1.0559946842364416,
  'jesDown': 0.9453685762165103,
  'jerUp': 1.0047304057268256,
  'jerDown': 0.9942523963376727},
 '2017': {'q2Down': 0.7283174919889528,
  'q2Up': 1.3717094436848096,
  'pdfUp': 1.0069648130757558,
  'pileupDown': 0.9965127866927616,
  'pileupUp': 1.0030809540288723,
  'btagUp': 1.0808531781601975,
  'btagDown': 0.9217395167716735,
  'prefiringUp': 0.9925275694493612,
  'pdfDown': 1.013929626151511,
  'prefiringDown': 1.0074316960192458,
  'jesUp': 1.0466347594197296,
  'jesDown': 0.9554617092349705,
  'jerUp': 1.008872409266884,
  'jerDown': 0.9918002175770939},
 '2018': {'q2Down': 0.7273804055118427,
  'btagUp': 1

In [12]:
wgts

{'2018': defaultdict_accumulator(float,
                         {'nominal': 575909620.4926904,
                          'q2Down': 418905373.2921446,
                          'btagUp': 620788731.4171407,
                          'pileupDown': 575071146.4011574,
                          'q2Up': 792160458.4992911,
                          'btagDown': 532396687.0790832,
                          'pileupUp': 576670569.2413595,
                          'pdfDown': 584040557.6326777,
                          'pdfUp': 579975089.062684,
                          'jesUp': 608515640.274138,
                          'jesDown': 545964335.2468631,
                          'jerUp': 579092142.7771422,
                          'jerDown': 572317751.4120592}),
 '2017': defaultdict_accumulator(float,
                         {'nominal': 408575327.31931365,
                          'q2Down': 297572557.681768,
                          'q2Up': 560446634.9405147,
                          'pdfUp':

In [13]:
table2 = '''
\\begin{table}
    \\begin{center}
        \\begin{tabular}{ |p{1cm}||p{3cm}| |p{3cm}| } \\hline 
            \\textbf{Year} & \\textbf{Systematic} & \\textbf{up/down} \\\\ \\hline
              2016   & PileUp       &   '''+'+{0:0.2f}/-{1:0.2f}'.format(wdata['2016']['pileupUp'], wdata['2016']['pileupDown'])+''' \\\\
                     & PDF          &   '''+'+{0:0.2f}/-{1:0.2f}'.format(wdata['2016']['pdfUp'],    wdata['2016']['pdfDown'])+''' \\\\ 
                     & $Q^2$        &   '''+'+{0:0.2f}/-{1:0.2f}'.format(wdata['2016']['q2Up'],     wdata['2016']['q2Down'])+''' \\\\ 
                     & btag         &   '''+'+{0:0.2f}/-{1:0.2f}'.format(wdata['2016']['btagUp'],   wdata['2016']['btagDown'])+''' \\\\ 
                     & JES          &   '''+'+{0:0.2f}/-{1:0.2f}'.format(wdata['2016']['jesUp'],    wdata['2016']['jesDown'])+''' \\\\ 
                     & JER          &   '''+'+{0:0.2f}/-{1:0.2f}'.format(wdata['2016']['jerUp'],    wdata['2016']['jerDown'])+''' \\\\
                     & L1 Prefiring &   '''+'+{0:0.2f}/-{1:0.2f}'.format(wdata['2017']['prefiringUp'],    wdata['2017']['prefiringDown'])+''' \\\\
              \\hline    
              2017   & PileUp       &   '''+'+{0:0.2f}/-{1:0.2f}'.format(wdata['2017']['pileupUp'], wdata['2017']['pileupDown'])+''' \\\\
                     & PDF          &   '''+'+{0:0.2f}/-{1:0.2f}'.format(wdata['2017']['pdfUp'],    wdata['2017']['pdfDown'])+''' \\\\ 
                     & $Q^2$        &   '''+'+{0:0.2f}/-{1:0.2f}'.format(wdata['2017']['q2Up'],     wdata['2017']['q2Down'])+''' \\\\ 
                     & btag         &   '''+'+{0:0.2f}/-{1:0.2f}'.format(wdata['2017']['btagUp'],   wdata['2017']['btagDown'])+''' \\\\ 
                     & JES          &   '''+'+{0:0.2f}/-{1:0.2f}'.format(wdata['2017']['jesUp'],    wdata['2017']['jesDown'])+''' \\\\ 
                     & JER          &   '''+'+{0:0.2f}/-{1:0.2f}'.format(wdata['2017']['jerUp'],    wdata['2017']['jerDown'])+''' \\\\
                     & L1 Prefiring &   '''+'+{0:0.2f}/-{1:0.2f}'.format(wdata['2017']['prefiringUp'],    wdata['2017']['prefiringDown'])+''' \\\\
              \\hline
              2018   & PileUp       &   '''+'+{0:0.2f}/-{1:0.2f}'.format(wdata['2018']['pileupUp'], wdata['2018']['pileupDown'])+''' \\\\
                     & PDF          &   '''+'+{0:0.2f}/-{1:0.2f}'.format(wdata['2018']['pdfUp'],    wdata['2018']['pdfDown'])+''' \\\\ 
                     & $Q^2$        &   '''+'+{0:0.2f}/-{1:0.2f}'.format(wdata['2018']['q2Up'],     wdata['2018']['q2Down'])+''' \\\\ 
                     & btag         &   '''+'+{0:0.2f}/-{1:0.2f}'.format(wdata['2018']['btagUp'],   wdata['2018']['btagDown'])+''' \\\\ 
                     & JES          &   '''+'+{0:0.2f}/-{1:0.2f}'.format(wdata['2018']['jesUp'],    wdata['2018']['jesDown'])+''' \\\\ 
                     & JER          &   '''+'+{0:0.2f}/-{1:0.2f}'.format(wdata['2018']['jerUp'],    wdata['2018']['jerDown'])+''' \\\\

               \\hline
        \\end{tabular}
        \\label{tab:syst}
        \\caption{Nuisance parameters for 2016, 2017, 2018 MC TTbar datasets}
    \\end{center}
\\end{table}

'''

In [14]:
print(table2)


\begin{table}
    \begin{center}
        \begin{tabular}{ |p{1cm}||p{3cm}| |p{3cm}| } \hline 
            \textbf{Year} & \textbf{Systematic} & \textbf{up/down} \\ \hline
              2016   & PileUp       &   +0.00/-0.00 \\
                     & PDF          &   +0.01/-0.01 \\ 
                     & $Q^2$        &   +0.38/-0.27 \\ 
                     & btag         &   +0.07/-0.07 \\ 
                     & JES          &   +0.06/-0.05 \\ 
                     & JER          &   +0.00/-0.01 \\
                     & L1 Prefiring &   +0.01/-0.01 \\
              \hline    
              2017   & PileUp       &   +0.00/-0.00 \\
                     & PDF          &   +0.01/-0.01 \\ 
                     & $Q^2$        &   +0.37/-0.27 \\ 
                     & btag         &   +0.08/-0.08 \\ 
                     & JES          &   +0.05/-0.04 \\ 
                     & JER          &   +0.01/-0.01 \\
                     & L1 Prefiring &   +0.01/-0.01 \\
              \hline
    

In [15]:
'+{0:0.2f}/-{1:0.2f}'.format(wdata['pileupUp'],wdata['pileupDown'])

KeyError: 'pileupUp'

## Event count comparisons

### raw events

In [17]:
# scenario1 = cmsv2 + csvv2
# scenario2 = cmsv2 + deepcsv
# scenario3 = deepak8 + deepcsv


# s1_file1 = util.load('../outputs/table/TTbar_2016_700to1000_cmstop_csvv2.coffea')
# s1_file2 = util.load('../outputs/table/TTbar_2016_1000toInf_cmstop_csvv2.coffea')
# s1_file3 = util.load('../outputs/table/TTbar_2016APV_700to1000_cmstop_csvv2.coffea')
# s1_file4 = util.load('../outputs/table/TTbar_2016APV_1000toInf_cmstop_csvv2.coffea')

s1_file1 = util.load('../outputs/table/TTbar_2016_700to1000_cmstop_csvv2_HT950.coffea')
s1_file2 = util.load('../outputs/table/TTbar_2016_1000toInf_cmstop_csvv2_HT950.coffea')
s1_file3 = util.load('../outputs/table/TTbar_2016APV_700to1000_cmstop_csvv2_HT950.coffea')
s1_file4 = util.load('../outputs/table/TTbar_2016APV_1000toInf_cmstop_csvv2_HT950.coffea')

s2_file1 = util.load('../outputs/table/TTbar_2016_700to1000_cmstop_deepcsv_HT950.coffea')
s2_file2 = util.load('../outputs/table/TTbar_2016_1000toInf_cmstop_deepcsv_HT950.coffea')
s2_file3 = util.load('../outputs/table/TTbar_2016APV_700to1000_cmstop_deepcsv_HT950.coffea')
s2_file4 = util.load('.z./outputs/table/TTbar_2016APV_1000toInf_cmstop_deepcsv_HT950.coffea')

s3_file1 = util.load('../outputs/table/TTbar_2016_700to1000_HT950.coffea')
s3_file2 = util.load('../outputs/table/TTbar_2016_1000toInf_HT950.coffea')
s3_file3 = util.load('../outputs/table/TTbar_2016APV_700to1000_HT950.coffea')
s3_file4 = util.load('../outputs/table/TTbar_2016APV_1000toInf_HT950.coffea')



# s1_file1 = util.load('../outputs/table/table/TTbar_2016_700to1000_cmstop_csvv2_HT950_test.coffea')
# s1_file2 = util.load('../outputs/table/table/TTbar_2016_1000toInf_cmstop_csvv2_HT950_test.coffea')
# s1_file3 = util.load('../outputs/table/table/TTbar_2016APV_700to1000_cmstop_csvv2_HT950_test.coffea')
# s1_file4 = util.load('../outputs/table/table/TTbar_2016APV_1000toInf_cmstop_csvv2_HT950_test.coffea')

# s2_file1 = util.load('../outputs/table/table/TTbar_2016_700to1000_cmstop_deepcsv_HT950_test.coffea')
# s2_file2 = util.load('../outputs/table/table/TTbar_2016_1000toInf_cmstop_deepcsv_HT950_test.coffea')
# s2_file3 = util.load('../outputs/table/table/TTbar_2016APV_700to1000_cmstop_deepcsv_HT950_test.coffea')
# s2_file4 = util.load('../outputs/table/table/TTbar_2016APV_1000toInf_cmstop_deepcsv_HT950_test.coffea')

# s3_file1 = util.load('../outputs/table/TTbar_2016_700to1000_HT950_test.coffea')
# s3_file2 = util.load('../outputs/table/TTbar_2016_1000toInf_HT950_test.coffea')
# s3_file3 = util.load('../outputs/table/TTbar_2016APV_700to1000_HT950_test.coffea')
# s3_file4 = util.load('../outputs/table/TTbar_2016APV_1000toInf_HT950_test.coffea')


    
label_map = s3_file1['analysisCategories']
label_to_int = {label: i for i, label in label_map.items()}
label_to_int
    
    
    


{'at0bcen': 0,
 'at0bfwd': 1,
 'at1bcen': 2,
 'at1bfwd': 3,
 'at2bcen': 4,
 'at2bfwd': 5,
 'pret0bcen': 6,
 'pret0bfwd': 7,
 'pret1bcen': 8,
 'pret1bfwd': 9,
 'pret2bcen': 10,
 'pret2bfwd': 11,
 '2t0bcen': 12,
 '2t0bfwd': 13,
 '2t1bcen': 14,
 '2t1bfwd': 15,
 '2t2bcen': 16,
 '2t2bfwd': 17}

In [27]:
ttbar1 = util.load('../local/history/medium1023/coffea/scale/TTbar_2016.coffea')
ttbar2 = util.load('../local/history/medium1023/coffea/scale/TTbar_2016APV.coffea')
ttbar3 = util.load('../local/history/medium1023/coffea/scale/TTbar_2017.coffea')
ttbar4 = util.load('../local/history/medium1023/coffea/scale/TTbar_2018.coffea')

data1 = util.load('../local/history/medium1023/coffea/scale/JetHT_2016.coffea')
data2 = util.load('../local/history/medium1023/coffea/scale/JetHT_2016APV.coffea')
data3 = util.load('../local/history/medium1023/coffea/scale/JetHT_2017.coffea')
data4 = util.load('../local/history/medium1023/coffea/scale/JetHT_2018.coffea')


### scaled events

In [18]:
functions.xs['TTbar']['700to1000'], functions.xs['TTbar']['1000toInf']

(76.605096, 20.5777424)

In [30]:
# s3_file1['cutflow']
data1['cutflow']

defaultdict_accumulator(int,
                        {'all events 1': 13227622,
                         'all events': 13155336,
                         'sumw': 13155336.0,
                         'sumw2': 13155336.0,
                         'trigger': 1691624,
                         'htCut': 1588058,
                         'metfilter': 1582958,
                         'jetid': 1582922,
                         'jetkincut': 1362508,
                         'twoFatJets': 857748,
                         'oneTTbar': 857748,
                         'dPhiCut': 855356,
                         'Good Subjets': 835788,
                         'at0bcen': 103036,
                         'at0bfwd': 112096,
                         'at1bcen': 30360,
                         'at1bfwd': 30552,
                         'at2bcen': 1980,
                         'at2bfwd': 1620,
                         'pret0bcen': 27064,
                         'pret0bfwd': 30680,
                      

## $N_{events}$

In [20]:
ttbar_2016_700to1000_events    = 33502717
ttbar_2016_1000toInf_events    = 23673116
ttbar_2016APV_700to1000_events = 23727154
ttbar_2016APV_1000toInf_events = 23208356



print(s1_file1['cutflow']['all events'] / ttbar_2016_700to1000_events)
print(s1_file2['cutflow']['all events'] / ttbar_2016_1000toInf_events)
print(s1_file3['cutflow']['all events'] / ttbar_2016APV_700to1000_events)
print(s1_file4['cutflow']['all events'] / ttbar_2016APV_1000toInf_events)

print(s2_file1['cutflow']['all events'] / ttbar_2016_700to1000_events)
print(s2_file2['cutflow']['all events'] / ttbar_2016_1000toInf_events)
print(s2_file3['cutflow']['all events'] / ttbar_2016APV_700to1000_events)
print(s2_file4['cutflow']['all events'] / ttbar_2016APV_1000toInf_events)

print(s3_file1['cutflow']['all events'] / ttbar_2016_700to1000_events)
print(s3_file2['cutflow']['all events'] / ttbar_2016_1000toInf_events)
print(s3_file3['cutflow']['all events'] / ttbar_2016APV_700to1000_events)
print(s3_file4['cutflow']['all events'] / ttbar_2016APV_1000toInf_events)

1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0


### scale factors

In [21]:
scaling = True

s1f1 = 1.0
s1f2 = 1.0
s1f3 = 1.0
s1f4 = 1.0

s2f1 = 1.0
s2f2 = 1.0
s2f3 = 1.0
s2f4 = 1.0

s3f1 = 1.0
s3f2 = 1.0
s3f3 = 1.0
s3f4 = 1.0

factor = 16

if scaling:
    
    s1f1 = functions.lumi['2016'] * functions.xs['TTbar']['700to1000'] / s1_file1['cutflow']['all events']
    s1f2 = functions.lumi['2016'] * functions.xs['TTbar']['1000toInf'] / s1_file2['cutflow']['all events']
    s1f3 = functions.lumi['2016APV'] * functions.xs['TTbar']['700to1000'] / s1_file3['cutflow']['all events']
    s1f4 = functions.lumi['2016APV'] * functions.xs['TTbar']['1000toInf'] / s1_file4['cutflow']['all events']

    s2f1 = functions.lumi['2016'] * functions.xs['TTbar']['700to1000'] / s2_file1['cutflow']['all events']
    s2f2 = functions.lumi['2016'] * functions.xs['TTbar']['1000toInf'] / s2_file2['cutflow']['all events']
    s2f3 = functions.lumi['2016APV'] * functions.xs['TTbar']['700to1000'] / s2_file3['cutflow']['all events']
    s2f4 = functions.lumi['2016APV'] * functions.xs['TTbar']['1000toInf'] / s2_file4['cutflow']['all events']
   
    s3f1 = functions.lumi['2016'] * functions.xs['TTbar']['700to1000'] / s3_file1['cutflow']['all events']
    s3f2 = functions.lumi['2016'] * functions.xs['TTbar']['1000toInf'] / s3_file2['cutflow']['all events']
    s3f3 = functions.lumi['2016APV'] * functions.xs['TTbar']['700to1000'] / s3_file3['cutflow']['all events']
    s3f4 = functions.lumi['2016APV'] * functions.xs['TTbar']['1000toInf'] / s3_file4['cutflow']['all events']

    
#     s1f1 = functions.lumi['2016'] * functions.xs['TTbar']['700to1000'] / s1_file1['cutflow']['sumw']
#     s1f2 = functions.lumi['2016'] * functions.xs['TTbar']['1000toInf'] / s1_file2['cutflow']['sumw']
#     s1f3 = functions.lumi['2016APV'] * functions.xs['TTbar']['700to1000'] / s1_file3['cutflow']['sumw']
#     s1f4 = functions.lumi['2016APV'] * functions.xs['TTbar']['1000toInf'] / s1_file4['cutflow']['sumw']

#     s2f1 = functions.lumi['2016'] * functions.xs['TTbar']['700to1000'] / s2_file1['cutflow']['sumw']
#     s2f2 = functions.lumi['2016'] * functions.xs['TTbar']['1000toInf'] / s2_file2['cutflow']['sumw']
#     s2f3 = functions.lumi['2016APV'] * functions.xs['TTbar']['700to1000'] / s2_file3['cutflow']['sumw']
#     s2f4 = functions.lumi['2016APV'] * functions.xs['TTbar']['1000toInf'] / s2_file4['cutflow']['sumw']
    

#     s3f1 = functions.lumi['2016'] * functions.xs['TTbar']['700to1000'] / s3_file1['cutflow']['sumw']
#     s3f2 = functions.lumi['2016'] * functions.xs['TTbar']['1000toInf'] / s3_file2['cutflow']['sumw']
#     s3f3 = functions.lumi['2016APV'] * functions.xs['TTbar']['700to1000'] / s3_file3['cutflow']['sumw']
#     s3f4 = functions.lumi['2016APV'] * functions.xs['TTbar']['1000toInf'] / s3_file4['cutflow']['sumw']

    
    
    
#     s1f1 = functions.lumi['2016'] * functions.xs['TTbar']['700to1000']    / ttbar_2016_700to1000_events   
#     s1f2 = functions.lumi['2016'] * functions.xs['TTbar']['1000toInf']    / ttbar_2016_1000toInf_events   
#     s1f3 = functions.lumi['2016APV'] * functions.xs['TTbar']['700to1000'] / ttbar_2016APV_700to1000_events
#     s1f4 = functions.lumi['2016APV'] * functions.xs['TTbar']['1000toInf'] / ttbar_2016APV_1000toInf_events

#     s2f1 = functions.lumi['2016'] * functions.xs['TTbar']['700to1000']    / ttbar_2016_700to1000_events   
#     s2f2 = functions.lumi['2016'] * functions.xs['TTbar']['1000toInf']    / ttbar_2016_1000toInf_events   
#     s2f3 = functions.lumi['2016APV'] * functions.xs['TTbar']['700to1000'] / ttbar_2016APV_700to1000_events
#     s2f4 = functions.lumi['2016APV'] * functions.xs['TTbar']['1000toInf'] / ttbar_2016APV_1000toInf_events
   
#     s3f1 = functions.lumi['2016'] * functions.xs['TTbar']['700to1000']    / ttbar_2016_700to1000_events   
#     s3f2 = functions.lumi['2016'] * functions.xs['TTbar']['1000toInf']    / ttbar_2016_1000toInf_events   
#     s3f3 = functions.lumi['2016APV'] * functions.xs['TTbar']['700to1000'] / ttbar_2016APV_700to1000_events
#     s3f4 = functions.lumi['2016APV'] * functions.xs['TTbar']['1000toInf'] / ttbar_2016APV_1000toInf_events





In [22]:
s3_file4['cutflow']

defaultdict_accumulator(int,
                        {'all events 1': 23208356,
                         'all events': 23208356,
                         'sumw': 15118461280.875,
                         'sumw2': 10955853821712.0,
                         'htCut': 4324485,
                         'jetid': 4323536,
                         'jetkincut': 3733294,
                         'twoFatJets': 2349569,
                         'oneTTbar': 2349569,
                         'dPhiCut': 2326290,
                         'Good Subjets': 2289142,
                         'at0bcen': 84590,
                         'at0bfwd': 67603,
                         'at1bcen': 185057,
                         'at1bfwd': 141657,
                         'at2bcen': 102432,
                         'at2bfwd': 73209,
                         'pret0bcen': 81535,
                         'pret0bfwd': 59909,
                         'pret1bcen': 324256,
                         'pret1bfwd': 231097,
    

In [23]:



s1_events_0bcen = np.sum([
    s1f1*np.sum(s1_file1['ttbarmass'][{'systematic':'nominal', 'anacat':label_to_int['2t0bcen']}].values()),
    s1f2*np.sum(s1_file2['ttbarmass'][{'systematic':'nominal', 'anacat':label_to_int['2t0bcen']}].values()),
    s1f3*np.sum(s1_file3['ttbarmass'][{'systematic':'nominal', 'anacat':label_to_int['2t0bcen']}].values()),
    s1f4*np.sum(s1_file4['ttbarmass'][{'systematic':'nominal', 'anacat':label_to_int['2t0bcen']}].values())
])


s1_events_1bcen = np.sum([
    s1f1*np.sum(s1_file1['ttbarmass'][{'systematic':'nominal', 'anacat':label_to_int['2t1bcen']}].values()),
    s1f2*np.sum(s1_file2['ttbarmass'][{'systematic':'nominal', 'anacat':label_to_int['2t1bcen']}].values()),
    s1f3*np.sum(s1_file3['ttbarmass'][{'systematic':'nominal', 'anacat':label_to_int['2t1bcen']}].values()),
    s1f4*np.sum(s1_file4['ttbarmass'][{'systematic':'nominal', 'anacat':label_to_int['2t1bcen']}].values())
])


s1_events_2bcen = np.sum([
    s1f1*np.sum(s1_file1['ttbarmass'][{'systematic':'nominal', 'anacat':label_to_int['2t2bcen']}].values()),
    s1f2*np.sum(s1_file2['ttbarmass'][{'systematic':'nominal', 'anacat':label_to_int['2t2bcen']}].values()),
    s1f3*np.sum(s1_file3['ttbarmass'][{'systematic':'nominal', 'anacat':label_to_int['2t2bcen']}].values()),
    s1f4*np.sum(s1_file4['ttbarmass'][{'systematic':'nominal', 'anacat':label_to_int['2t2bcen']}].values())
])


s1_events_0bfwd = np.sum([
    s1f1*np.sum(s1_file1['ttbarmass'][{'systematic':'nominal', 'anacat':label_to_int['2t0bfwd']}].values()),
    s1f2*np.sum(s1_file2['ttbarmass'][{'systematic':'nominal', 'anacat':label_to_int['2t0bfwd']}].values()),
    s1f3*np.sum(s1_file3['ttbarmass'][{'systematic':'nominal', 'anacat':label_to_int['2t0bfwd']}].values()),
    s1f4*np.sum(s1_file4['ttbarmass'][{'systematic':'nominal', 'anacat':label_to_int['2t0bfwd']}].values())
])


s1_events_1bfwd = np.sum([
    s1f1*np.sum(s1_file1['ttbarmass'][{'systematic':'nominal', 'anacat':label_to_int['2t1bfwd']}].values()),
    s1f2*np.sum(s1_file2['ttbarmass'][{'systematic':'nominal', 'anacat':label_to_int['2t1bfwd']}].values()),
    s1f3*np.sum(s1_file3['ttbarmass'][{'systematic':'nominal', 'anacat':label_to_int['2t1bfwd']}].values()),
    s1f4*np.sum(s1_file4['ttbarmass'][{'systematic':'nominal', 'anacat':label_to_int['2t1bfwd']}].values())
])


s1_events_2bfwd = np.sum([
    s1f1*np.sum(s1_file1['ttbarmass'][{'systematic':'nominal', 'anacat':label_to_int['2t2bfwd']}].values()),
    s1f2*np.sum(s1_file2['ttbarmass'][{'systematic':'nominal', 'anacat':label_to_int['2t2bfwd']}].values()),
    s1f3*np.sum(s1_file3['ttbarmass'][{'systematic':'nominal', 'anacat':label_to_int['2t2bfwd']}].values()),
    s1f4*np.sum(s1_file4['ttbarmass'][{'systematic':'nominal', 'anacat':label_to_int['2t2bfwd']}].values())
])






s2_events_0bcen = np.sum([
    s2f1*np.sum(s2_file1['ttbarmass'][{'systematic':'nominal', 'anacat':label_to_int['2t0bcen']}].values()),
    s2f2*np.sum(s2_file2['ttbarmass'][{'systematic':'nominal', 'anacat':label_to_int['2t0bcen']}].values()),
    s2f3*np.sum(s2_file3['ttbarmass'][{'systematic':'nominal', 'anacat':label_to_int['2t0bcen']}].values()),
    s2f4*np.sum(s2_file4['ttbarmass'][{'systematic':'nominal', 'anacat':label_to_int['2t0bcen']}].values())
])


s2_events_1bcen = np.sum([
    s2f1*np.sum(s2_file1['ttbarmass'][{'systematic':'nominal', 'anacat':label_to_int['2t1bcen']}].values()),
    s2f2*np.sum(s2_file2['ttbarmass'][{'systematic':'nominal', 'anacat':label_to_int['2t1bcen']}].values()),
    s2f3*np.sum(s2_file3['ttbarmass'][{'systematic':'nominal', 'anacat':label_to_int['2t1bcen']}].values()),
    s2f4*np.sum(s2_file4['ttbarmass'][{'systematic':'nominal', 'anacat':label_to_int['2t1bcen']}].values())
])


s2_events_2bcen = np.sum([
    s2f1*np.sum(s2_file1['ttbarmass'][{'systematic':'nominal', 'anacat':label_to_int['2t2bcen']}].values()),
    s2f2*np.sum(s2_file2['ttbarmass'][{'systematic':'nominal', 'anacat':label_to_int['2t2bcen']}].values()),
    s2f3*np.sum(s2_file3['ttbarmass'][{'systematic':'nominal', 'anacat':label_to_int['2t2bcen']}].values()),
    s2f4*np.sum(s2_file4['ttbarmass'][{'systematic':'nominal', 'anacat':label_to_int['2t2bcen']}].values())
])


s2_events_0bfwd = np.sum([
    s2f1*np.sum(s2_file1['ttbarmass'][{'systematic':'nominal', 'anacat':label_to_int['2t0bfwd']}].values()),
    s2f2*np.sum(s2_file2['ttbarmass'][{'systematic':'nominal', 'anacat':label_to_int['2t0bfwd']}].values()),
    s2f3*np.sum(s2_file3['ttbarmass'][{'systematic':'nominal', 'anacat':label_to_int['2t0bfwd']}].values()),
    s2f4*np.sum(s2_file4['ttbarmass'][{'systematic':'nominal', 'anacat':label_to_int['2t0bfwd']}].values())
])


s2_events_1bfwd = np.sum([
    s2f1*np.sum(s2_file1['ttbarmass'][{'systematic':'nominal', 'anacat':label_to_int['2t1bfwd']}].values()),
    s2f2*np.sum(s2_file2['ttbarmass'][{'systematic':'nominal', 'anacat':label_to_int['2t1bfwd']}].values()),
    s2f3*np.sum(s2_file3['ttbarmass'][{'systematic':'nominal', 'anacat':label_to_int['2t1bfwd']}].values()),
    s2f4*np.sum(s2_file4['ttbarmass'][{'systematic':'nominal', 'anacat':label_to_int['2t1bfwd']}].values())
])


s2_events_2bfwd = np.sum([
    s2f1*np.sum(s2_file1['ttbarmass'][{'systematic':'nominal', 'anacat':label_to_int['2t2bfwd']}].values()),
    s2f2*np.sum(s2_file2['ttbarmass'][{'systematic':'nominal', 'anacat':label_to_int['2t2bfwd']}].values()),
    s2f3*np.sum(s2_file3['ttbarmass'][{'systematic':'nominal', 'anacat':label_to_int['2t2bfwd']}].values()),
    s2f4*np.sum(s2_file4['ttbarmass'][{'systematic':'nominal', 'anacat':label_to_int['2t2bfwd']}].values())
])








s3_events_0bcen = np.sum([
    s3f1*np.sum(s3_file1['ttbarmass'][{'systematic':'nominal', 'anacat':label_to_int['2t0bcen']}].values()),
    s3f2*np.sum(s3_file2['ttbarmass'][{'systematic':'nominal', 'anacat':label_to_int['2t0bcen']}].values()),
    s3f3*np.sum(s3_file3['ttbarmass'][{'systematic':'nominal', 'anacat':label_to_int['2t0bcen']}].values()),
    s3f4*np.sum(s3_file4['ttbarmass'][{'systematic':'nominal', 'anacat':label_to_int['2t0bcen']}].values())
])


s3_events_1bcen = np.sum([
    s3f1*np.sum(s3_file1['ttbarmass'][{'systematic':'nominal', 'anacat':label_to_int['2t1bcen']}].values()),
    s3f2*np.sum(s3_file2['ttbarmass'][{'systematic':'nominal', 'anacat':label_to_int['2t1bcen']}].values()),
    s3f3*np.sum(s3_file3['ttbarmass'][{'systematic':'nominal', 'anacat':label_to_int['2t1bcen']}].values()),
    s3f4*np.sum(s3_file4['ttbarmass'][{'systematic':'nominal', 'anacat':label_to_int['2t1bcen']}].values())
])


s3_events_2bcen = np.sum([
    s3f1*np.sum(s3_file1['ttbarmass'][{'systematic':'nominal', 'anacat':label_to_int['2t2bcen']}].values()),
    s3f2*np.sum(s3_file2['ttbarmass'][{'systematic':'nominal', 'anacat':label_to_int['2t2bcen']}].values()),
    s3f3*np.sum(s3_file3['ttbarmass'][{'systematic':'nominal', 'anacat':label_to_int['2t2bcen']}].values()),
    s3f4*np.sum(s3_file4['ttbarmass'][{'systematic':'nominal', 'anacat':label_to_int['2t2bcen']}].values())
])


s3_events_0bfwd = np.sum([
    s3f1*np.sum(s3_file1['ttbarmass'][{'systematic':'nominal', 'anacat':label_to_int['2t0bfwd']}].values()),
    s3f2*np.sum(s3_file2['ttbarmass'][{'systematic':'nominal', 'anacat':label_to_int['2t0bfwd']}].values()),
    s3f3*np.sum(s3_file3['ttbarmass'][{'systematic':'nominal', 'anacat':label_to_int['2t0bfwd']}].values()),
    s3f4*np.sum(s3_file4['ttbarmass'][{'systematic':'nominal', 'anacat':label_to_int['2t0bfwd']}].values())
])


s3_events_1bfwd = np.sum([
    s3f1*np.sum(s3_file1['ttbarmass'][{'systematic':'nominal', 'anacat':label_to_int['2t1bfwd']}].values()),
    s3f2*np.sum(s3_file2['ttbarmass'][{'systematic':'nominal', 'anacat':label_to_int['2t1bfwd']}].values()),
    s3f3*np.sum(s3_file3['ttbarmass'][{'systematic':'nominal', 'anacat':label_to_int['2t1bfwd']}].values()),
    s3f4*np.sum(s3_file4['ttbarmass'][{'systematic':'nominal', 'anacat':label_to_int['2t1bfwd']}].values())
])


s3_events_2bfwd = np.sum([
    s3f1*np.sum(s3_file1['ttbarmass'][{'systematic':'nominal', 'anacat':label_to_int['2t2bfwd']}].values()),
    s3f2*np.sum(s3_file2['ttbarmass'][{'systematic':'nominal', 'anacat':label_to_int['2t2bfwd']}].values()),
    s3f3*np.sum(s3_file3['ttbarmass'][{'systematic':'nominal', 'anacat':label_to_int['2t2bfwd']}].values()),
    s3f4*np.sum(s3_file4['ttbarmass'][{'systematic':'nominal', 'anacat':label_to_int['2t2bfwd']}].values())
])


s1_events = s1_events_0bcen + s1_events_1bcen + s1_events_2bcen + s1_events_0bfwd + s1_events_1bfwd + s1_events_2bfwd
s2_events = s2_events_0bcen + s2_events_1bcen + s2_events_2bcen + s2_events_0bfwd + s2_events_1bfwd + s2_events_2bfwd
s3_events = s3_events_0bcen + s3_events_1bcen + s3_events_2bcen + s3_events_0bfwd + s3_events_1bfwd + s3_events_2bfwd





In [33]:
data_events_2bfwd = np.sum([
    np.sum(data1['ttbarmass'][{'systematic':'nominal', 'anacat':label_to_int['2t2bfwd']}].values()),
    np.sum(data2['ttbarmass'][{'systematic':'nominal', 'anacat':label_to_int['2t2bfwd']}].values()),
    np.sum(data3['ttbarmass'][{'systematic':'nominal', 'anacat':label_to_int['2t2bfwd']}].values()),
    np.sum(data4['ttbarmass'][{'systematic':'nominal', 'anacat':label_to_int['2t2bfwd']}].values())
])


data_events_2bcen = np.sum([
    np.sum(data1['ttbarmass'][{'systematic':'nominal', 'anacat':label_to_int['2t2bcen']}].values()),
    np.sum(data2['ttbarmass'][{'systematic':'nominal', 'anacat':label_to_int['2t2bcen']}].values()),
    np.sum(data3['ttbarmass'][{'systematic':'nominal', 'anacat':label_to_int['2t2bcen']}].values()),
    np.sum(data4['ttbarmass'][{'systematic':'nominal', 'anacat':label_to_int['2t2bcen']}].values())
])

data_events_1bfwd = np.sum([
    np.sum(data1['ttbarmass'][{'systematic':'nominal', 'anacat':label_to_int['2t1bfwd']}].values()),
    np.sum(data2['ttbarmass'][{'systematic':'nominal', 'anacat':label_to_int['2t1bfwd']}].values()),
    np.sum(data3['ttbarmass'][{'systematic':'nominal', 'anacat':label_to_int['2t1bfwd']}].values()),
    np.sum(data4['ttbarmass'][{'systematic':'nominal', 'anacat':label_to_int['2t1bfwd']}].values())
])

data_events_1bcen = np.sum([
    np.sum(data1['ttbarmass'][{'systematic':'nominal', 'anacat':label_to_int['2t1bcen']}].values()),
    np.sum(data2['ttbarmass'][{'systematic':'nominal', 'anacat':label_to_int['2t1bcen']}].values()),
    np.sum(data3['ttbarmass'][{'systematic':'nominal', 'anacat':label_to_int['2t1bcen']}].values()),
    np.sum(data4['ttbarmass'][{'systematic':'nominal', 'anacat':label_to_int['2t1bcen']}].values())
])

data_events_0bfwd = np.sum([
    np.sum(data1['ttbarmass'][{'systematic':'nominal', 'anacat':label_to_int['2t0bfwd']}].values()),
    np.sum(data2['ttbarmass'][{'systematic':'nominal', 'anacat':label_to_int['2t0bfwd']}].values()),
    np.sum(data3['ttbarmass'][{'systematic':'nominal', 'anacat':label_to_int['2t0bfwd']}].values()),
    np.sum(data4['ttbarmass'][{'systematic':'nominal', 'anacat':label_to_int['2t0bfwd']}].values())
])

data_events_0bcen = np.sum([
    np.sum(data1['ttbarmass'][{'systematic':'nominal', 'anacat':label_to_int['2t0bcen']}].values()),
    np.sum(data2['ttbarmass'][{'systematic':'nominal', 'anacat':label_to_int['2t0bcen']}].values()),
    np.sum(data3['ttbarmass'][{'systematic':'nominal', 'anacat':label_to_int['2t0bcen']}].values()),
    np.sum(data4['ttbarmass'][{'systematic':'nominal', 'anacat':label_to_int['2t0bcen']}].values())
])



In [34]:
ttbar_events_2bfwd = np.sum([
    np.sum(ttbar1['ttbarmass'][{'systematic':'nominal', 'anacat':label_to_int['2t2bfwd']}].values()),
    np.sum(ttbar2['ttbarmass'][{'systematic':'nominal', 'anacat':label_to_int['2t2bfwd']}].values()),
    np.sum(ttbar3['ttbarmass'][{'systematic':'nominal', 'anacat':label_to_int['2t2bfwd']}].values()),
    np.sum(ttbar4['ttbarmass'][{'systematic':'nominal', 'anacat':label_to_int['2t2bfwd']}].values())
])


ttbar_events_2bcen = np.sum([
    np.sum(ttbar1['ttbarmass'][{'systematic':'nominal', 'anacat':label_to_int['2t2bcen']}].values()),
    np.sum(ttbar2['ttbarmass'][{'systematic':'nominal', 'anacat':label_to_int['2t2bcen']}].values()),
    np.sum(ttbar3['ttbarmass'][{'systematic':'nominal', 'anacat':label_to_int['2t2bcen']}].values()),
    np.sum(ttbar4['ttbarmass'][{'systematic':'nominal', 'anacat':label_to_int['2t2bcen']}].values())
])

ttbar_events_1bfwd = np.sum([
    np.sum(ttbar1['ttbarmass'][{'systematic':'nominal', 'anacat':label_to_int['2t1bfwd']}].values()),
    np.sum(ttbar2['ttbarmass'][{'systematic':'nominal', 'anacat':label_to_int['2t1bfwd']}].values()),
    np.sum(ttbar3['ttbarmass'][{'systematic':'nominal', 'anacat':label_to_int['2t1bfwd']}].values()),
    np.sum(ttbar4['ttbarmass'][{'systematic':'nominal', 'anacat':label_to_int['2t1bfwd']}].values())
])

ttbar_events_1bcen = np.sum([
    np.sum(ttbar1['ttbarmass'][{'systematic':'nominal', 'anacat':label_to_int['2t1bcen']}].values()),
    np.sum(ttbar2['ttbarmass'][{'systematic':'nominal', 'anacat':label_to_int['2t1bcen']}].values()),
    np.sum(ttbar3['ttbarmass'][{'systematic':'nominal', 'anacat':label_to_int['2t1bcen']}].values()),
    np.sum(ttbar4['ttbarmass'][{'systematic':'nominal', 'anacat':label_to_int['2t1bcen']}].values())
])

ttbar_events_0bfwd = np.sum([
    np.sum(ttbar1['ttbarmass'][{'systematic':'nominal', 'anacat':label_to_int['2t0bfwd']}].values()),
    np.sum(ttbar2['ttbarmass'][{'systematic':'nominal', 'anacat':label_to_int['2t0bfwd']}].values()),
    np.sum(ttbar3['ttbarmass'][{'systematic':'nominal', 'anacat':label_to_int['2t0bfwd']}].values()),
    np.sum(ttbar4['ttbarmass'][{'systematic':'nominal', 'anacat':label_to_int['2t0bfwd']}].values())
])

ttbar_events_0bcen = np.sum([
    np.sum(ttbar1['ttbarmass'][{'systematic':'nominal', 'anacat':label_to_int['2t0bcen']}].values()),
    np.sum(ttbar2['ttbarmass'][{'systematic':'nominal', 'anacat':label_to_int['2t0bcen']}].values()),
    np.sum(ttbar3['ttbarmass'][{'systematic':'nominal', 'anacat':label_to_int['2t0bcen']}].values()),
    np.sum(ttbar4['ttbarmass'][{'systematic':'nominal', 'anacat':label_to_int['2t0bcen']}].values())
])



In [37]:

s1_events_0bcen = np.sum([
    s1f1*s1_file1['cutflow']['2t0bcen'],
    s1f2*s1_file2['cutflow']['2t0bcen'],
    s1f3*s1_file3['cutflow']['2t0bcen'],
    s1f4*s1_file4['cutflow']['2t0bcen']
])


s1_events_1bcen = np.sum([
    s1f1*s1_file1['cutflow']['2t1bcen'],
    s1f2*s1_file2['cutflow']['2t1bcen'],
    s1f3*s1_file3['cutflow']['2t1bcen'],
    s1f4*s1_file4['cutflow']['2t1bcen']
])

s1_events_2bcen = np.sum([
    s1f1*s1_file1['cutflow']['2t2bcen'],
    s1f2*s1_file2['cutflow']['2t2bcen'],
    s1f3*s1_file3['cutflow']['2t2bcen'],
    s1f4*s1_file4['cutflow']['2t2bcen']
])


s1_events_0bfwd = np.sum([
    s1f1*s1_file1['cutflow']['2t0bfwd'],
    s1f2*s1_file2['cutflow']['2t0bfwd'],
    s1f3*s1_file3['cutflow']['2t0bfwd'],
    s1f4*s1_file4['cutflow']['2t0bfwd']
])


s1_events_1bfwd = np.sum([
    s1f1*s1_file1['cutflow']['2t1bfwd'],
    s1f2*s1_file2['cutflow']['2t1bfwd'],
    s1f3*s1_file3['cutflow']['2t1bfwd'],
    s1f4*s1_file4['cutflow']['2t1bfwd']
])

s1_events_2bfwd = np.sum([
    s1f1*s1_file1['cutflow']['2t2bfwd'],
    s1f2*s1_file2['cutflow']['2t2bfwd'],
    s1f3*s1_file3['cutflow']['2t2bfwd'],
    s1f4*s1_file4['cutflow']['2t2bfwd']
])


s1_events_cutflow = np.sum([
    s1f1*s1_file1['cutflow']['Good Subjets'],
    s1f2*s1_file2['cutflow']['Good Subjets'],
    s1f3*s1_file3['cutflow']['Good Subjets'],
    s1f4*s1_file4['cutflow']['Good Subjets']
])


s2_events_0bcen = np.sum([
    s2f1*s2_file1['cutflow']['2t0bcen'],
    s2f2*s2_file2['cutflow']['2t0bcen'],
    s2f3*s2_file3['cutflow']['2t0bcen'],
    s2f4*s2_file4['cutflow']['2t0bcen']
])


s2_events_1bcen = np.sum([
    s2f1*s2_file1['cutflow']['2t1bcen'],
    s2f2*s2_file2['cutflow']['2t1bcen'],
    s2f3*s2_file3['cutflow']['2t1bcen'],
    s2f4*s2_file4['cutflow']['2t1bcen']
])

s2_events_2bcen = np.sum([
    s2f1*s2_file1['cutflow']['2t2bcen'],
    s2f2*s2_file2['cutflow']['2t2bcen'],
    s2f3*s2_file3['cutflow']['2t2bcen'],
    s2f4*s2_file4['cutflow']['2t2bcen']
])


s2_events_0bfwd = np.sum([
    s2f1*s2_file1['cutflow']['2t0bfwd'],
    s2f2*s2_file2['cutflow']['2t0bfwd'],
    s2f3*s2_file3['cutflow']['2t0bfwd'],
    s2f4*s2_file4['cutflow']['2t0bfwd']
])


s2_events_1bfwd = np.sum([
    s2f1*s2_file1['cutflow']['2t1bfwd'],
    s2f2*s2_file2['cutflow']['2t1bfwd'],
    s2f3*s2_file3['cutflow']['2t1bfwd'],
    s2f4*s2_file4['cutflow']['2t1bfwd']
])

s2_events_2bfwd = np.sum([
    s2f1*s2_file1['cutflow']['2t2bfwd'],
    s2f2*s2_file2['cutflow']['2t2bfwd'],
    s2f3*s2_file3['cutflow']['2t2bfwd'],
    s2f4*s2_file4['cutflow']['2t2bfwd']
])


s2_events_cutflow = np.sum([
    s2f1*s2_file1['cutflow']['Good Subjets'],
    s2f2*s2_file2['cutflow']['Good Subjets'],
    s2f3*s2_file3['cutflow']['Good Subjets'],
    s2f4*s2_file4['cutflow']['Good Subjets']
])



s3_events_0bcen = np.sum([
    s3f1*s3_file1['cutflow']['2t0bcen'],
    s3f2*s3_file2['cutflow']['2t0bcen'],
    s3f3*s3_file3['cutflow']['2t0bcen'],
    s3f4*s3_file4['cutflow']['2t0bcen']
])


s3_events_1bcen = np.sum([
    s3f1*s3_file1['cutflow']['2t1bcen'],
    s3f2*s3_file2['cutflow']['2t1bcen'],
    s3f3*s3_file3['cutflow']['2t1bcen'],
    s3f4*s3_file4['cutflow']['2t1bcen']
])

s3_events_2bcen = np.sum([
    s3f1*s3_file1['cutflow']['2t2bcen'],
    s3f2*s3_file2['cutflow']['2t2bcen'],
    s3f3*s3_file3['cutflow']['2t2bcen'],
    s3f4*s3_file4['cutflow']['2t2bcen']
])


s3_events_0bfwd = np.sum([
    s3f1*s3_file1['cutflow']['2t0bfwd'],
    s3f2*s3_file2['cutflow']['2t0bfwd'],
    s3f3*s3_file3['cutflow']['2t0bfwd'],
    s3f4*s3_file4['cutflow']['2t0bfwd']
])


s3_events_1bfwd = np.sum([
    s3f1*s3_file1['cutflow']['2t1bfwd'],
    s3f2*s3_file2['cutflow']['2t1bfwd'],
    s3f3*s3_file3['cutflow']['2t1bfwd'],
    s3f4*s3_file4['cutflow']['2t1bfwd']
])

s3_events_2bfwd = np.sum([
    s3f1*s3_file1['cutflow']['2t2bfwd'],
    s3f2*s3_file2['cutflow']['2t2bfwd'],
    s3f3*s3_file3['cutflow']['2t2bfwd'],
    s3f4*s3_file4['cutflow']['2t2bfwd']
])


s3_events_cutflow = np.sum([
    s3f1*s3_file1['cutflow']['Good Subjets'],
    s3f2*s3_file2['cutflow']['Good Subjets'],
    s3f3*s3_file3['cutflow']['Good Subjets'],
    s3f4*s3_file4['cutflow']['Good Subjets']
])




s1_events = s1_events_0bcen + s1_events_1bcen + s1_events_2bcen + s1_events_0bfwd + s1_events_1bfwd + s1_events_2bfwd
s2_events = s2_events_0bcen + s2_events_1bcen + s2_events_2bcen + s2_events_0bfwd + s2_events_1bfwd + s2_events_2bfwd
s3_events = s3_events_0bcen + s3_events_1bcen + s3_events_2bcen + s3_events_0bfwd + s3_events_1bfwd + s3_events_2bfwd
ttbar_events = ttbar_events_0bcen + ttbar_events_1bcen + ttbar_events_2bcen + ttbar_events_0bfwd + ttbar_events_1bfwd + ttbar_events_2bfwd
data_events = data_events_0bcen + data_events_1bcen + data_events_2bcen + data_events_0bfwd + data_events_1bfwd + data_events_2bfwd






In [25]:
print(s3_events, s3_events_cutflow)
print(s3_events_cutflow / s3_events)

21911.484198649214 102237.06429653845
4.665912330249226


In [40]:
string = '''

\\begin{table}
    \\resizebox{0.8\\textwidth}{!}{%
        \\begin{tabular}{@{}ccccccc@{}}
        & \\textbf{Category} & \\textbf{2016 AN} & \\textbf{CMSv2 CSVv2} & \\textbf{CMSv2 DeepCSV} & \\textbf{DeepAK8 0.1% DeepCSV} & \\textbf{DeepAK8 0.5% DeepCSV} \\\\ \\cline{1-6}
        \\multirow{6}{*}{Obs. Data}
        & 0bcen & $      22313$ & $      22447$ & $      20084$ & $            20661$ & $\\ \\  ''' + '  {a:0.0f}$ '.format(a=data_events_0bcen) + ''' \\\\ 
        & 0bfwd & $      21836$ & $      21767$ & $      19588$ & $            23187$ & $\\ \\  ''' + '  {a:0.0f}$ '.format(a=data_events_0bfwd) + ''' \\\\ \\cline{2-6}
        & 1bcen & $\\ \\    9368$ & $\\ \\    8936$ & $      10195$ & $        20581$ & $\\ \\  ''' + '  {a:0.0f}$ '.format(a=data_events_1bcen) + ''' \\\\ 
        & 1bfwd & $\\ \\    7973$ & $\\ \\    7667$ & $\\ \\    9178$ & $      20220$ & $\\ \\  ''' + '  {a:0.0f}$ '.format(a=data_events_1bfwd) + ''' \\\\ \\cline{2-6}
        & 2bcen & $\\ \\    2226$ & $\\ \\    2197$ & $\\ \\    3301$ & $\\ \\  6338$ & $\\ \\  ''' + '  {a:0.0f}$ '.format(a=data_events_2bcen) + ''' \\\\ 
        & 2bfwd & $\\ \\    1376$ & $\\ \\    1390$ & $\\ \\    2058$ & $\\ \\  5090$ & $\\ \\  ''' + '  {a:0.0f}$ '.format(a=data_events_2bfwd) + ''' \\\\ \\cline{1-6}\\\\\\cline{1-6}
        \\multirow{6}{*}{Tot. Bkg.}
        & 0bcen & $      22530$ & $      21258$ & $      19596$ & $      16370$ \\\\ 
        & 0bfwd & $      21388$ & $      20353$ & $      19270$ & $      18680$ \\\\ \\cline{2-6}
        & 1bcen & $\\ \\    9883$ & $\\ \\    8660$ & $      10286$ & $      22285$ \\\\ 
        & 1bfwd & $\\ \\    8227$ & $\\ \\    7268$ & $\\ \\    9148$ & $      22284$ \\\\ \\cline{2-6}
        & 2bcen & $\\ \\    2646$ & $\\ \\    2438$ & $\\ \\    3051$ & $\\ \\    4699$ \\\\ 
        & 2bfwd & $\\ \\    1674$ & $\\ \\    1515$ & $\\ \\    2020$ & $\\ \\    3995$ \\\\ \\cline{1-6}\\\\\\cline{1-6}
        \\multirow{7}{*}{SM $t\\bar{t}$}
        & 0bcen & $\\ \\  ''' + '  1261$  & $\\ \\    {a:0.0f}$ & $\\ \\    {b:0.0f}$ & $\\ \\    {c:0.0f}$ \\\\ & $\\ \\    {d:0.0f}$ \\\\ '.format(a=s1_events_0bcen, b=s2_events_0bcen, c=s3_events_0bcen, d=ttbar_events_0bcen) + '''
        & 0bfwd & $\\ \\  ''' + '   812$  & $\\ \\    {a:0.0f}$ & $\\ \\    {b:0.0f}$ & $\\ \\    {c:0.0f}$ \\\\ & $\\ \\    {d:0.0f}$ \\\\ '.format(a=s1_events_0bfwd, b=s2_events_0bfwd, c=s3_events_0bfwd, d=ttbar_events_0bfwd) + '''
        & 1bcen & $\\ \\  ''' + '  3347$  & $\\ \\    {a:0.0f}$ & $\\ \\    {b:0.0f}$ & $\\ \\    {c:0.0f}$ \\\\ & $\\ \\    {d:0.0f}$ \\\\ '.format(a=s1_events_1bcen, b=s2_events_1bcen, c=s3_events_1bcen, d=ttbar_events_1bcen) + '''
        & 1bfwd & $\\ \\  ''' + '  1980$  & $\\ \\    {a:0.0f}$ & $\\ \\    {b:0.0f}$ & $\\ \\    {c:0.0f}$ \\\\ & $\\ \\    {d:0.0f}$ \\\\ '.format(a=s1_events_1bfwd, b=s2_events_1bfwd, c=s3_events_1bfwd, d=ttbar_events_1bfwd) + '''
        & 2bcen & $\\ \\  ''' + '  2165$  & $\\ \\    {a:0.0f}$ & $\\ \\    {b:0.0f}$ & $\\ \\    {c:0.0f}$ \\\\ & $\\ \\    {d:0.0f}$ \\\\ '.format(a=s1_events_2bcen, b=s2_events_2bcen, c=s3_events_2bcen, d=ttbar_events_2bcen) + '''
        & 2bfwd & $\\ \\  ''' + '  1205$  & $\\ \\    {a:0.0f}$ & $\\ \\    {b:0.0f}$ & $\\ \\    {c:0.0f}$ \\\\ & $\\ \\    {d:0.0f}$ \\\\ '.format(a=s1_events_2bfwd, b=s2_events_2bfwd, c=s3_events_2bfwd, d=ttbar_events_2bfwd) + '''
        & Total & $\\ \\  ''' + '  10770$ & $\\ \\    {a:0.0f}$ & $\\ \\   {b:0.0f}$ & $\\ \\    {c:0.0f}$ \\\\ & $\\ \\    {d:0.0f}$ \\\\ '.format(a=s1_events, b=s2_events, c=s3_events, d=ttbar_events) + '''
        
      \\end{tabular}
      \\caption{Comparison of event counts between the older analysis ({\\bf 2016 AN}) and this analysis, using the old taggers
        ({\\bf CMSv2 \\& CSVv2}) and the newest criteria used in this analysis ({\\bf DeepAK8 \\& DeepCSV}).
        {\\it Note: Our background estimate and $t\\bar{t}$ event counts were taken pre-fit. The NTMJ component of the total background was estimated by using mistag rates, rather than "pass/fail functions".Also, we apply a NLO correction to $t\\bar{t}$ (which was not present in the 2016 analysis) when using one of the latest taggers.}}
      \\label{Table1_2016}
\\end{table}


'''
print(string)






\begin{table}
    \resizebox{0.8\textwidth}{!}{%
        \begin{tabular}{@{}ccccccc@{}}
        & \textbf{Category} & \textbf{2016 AN} & \textbf{CMSv2 CSVv2} & \textbf{CMSv2 DeepCSV} & \textbf{DeepAK8 0.1% DeepCSV} & \textbf{DeepAK8 0.5% DeepCSV} \\ \cline{1-6}
        \multirow{6}{*}{Obs. Data}
        & 0bcen & $      22313$ & $      22447$ & $      20084$ & $            20661$ & $\ \    205762$  \\ 
        & 0bfwd & $      21836$ & $      21767$ & $      19588$ & $            23187$ & $\ \    222598$  \\ \cline{2-6}
        & 1bcen & $\ \    9368$ & $\ \    8936$ & $      10195$ & $        20581$ & $\ \    171916$  \\ 
        & 1bfwd & $\ \    7973$ & $\ \    7667$ & $\ \    9178$ & $      20220$ & $\ \    158012$  \\ \cline{2-6}
        & 2bcen & $\ \    2226$ & $\ \    2197$ & $\ \    3301$ & $\ \  6338$ & $\ \    52960$  \\ 
        & 2bfwd & $\ \    1376$ & $\ \    1390$ & $\ \    2058$ & $\ \  5090$ & $\ \    37054$  \\ \cline{1-6}\\\cline{1-6}
        \multirow{6}{*}{Tot. 

## Parameters

In [ ]:
params_2016 = json.load(open('../data/corrections/params_2016.json'))
params_2017 = json.load(open('../data/corrections/params_2017.json'))
params_2018 = json.load(open('../data/corrections/params_2018.json'))


In [ ]:
params_2017['0bcen']

In [ ]:
param_table = '''
\\begin{table}[htbp]
    \\centering
        \\label{tab:syst}
       \\caption{Values for parameters of the 2016 transfer function}
        \\begin{tabular}{|c||c||c||c||c|} \\hline 
            \\textbf{Year} & \\textbf{Region} & \\textbf{Parameter} & \\textbf{Value} & \\textbf{Uncertainty} \\\\ \\hline
            \\multirow{20}{*}{2016}
              
                     & \\multirow{2}{*}{cen0b}  '''+'& p0 & {0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2016['0bcen']['param'][0], params_2016['0bcen']['error'][0])+'''
                     '''+'&       & p1 &  {0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2016['0bcen']['param'][1], params_2016['0bcen']['error'][1])+''' \\cline{2-5}
                     
                     & \\multirow{4}{*}{cen1b}  '''+'& p0 & {0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2016['1bcen']['param'][0], params_2016['1bcen']['error'][0])+''' 
                     '''+'&       & p1 & {0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2016['1bcen']['param'][1], params_2016['1bcen']['error'][1])+''' 
                     '''+'&       & p2 & {0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2016['1bcen']['param'][2], params_2016['1bcen']['error'][2])+'''
                     '''+'&       & p3 & {0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2016['1bcen']['param'][3], params_2016['1bcen']['error'][3])+''' \\cline{2-5}
                     
                     & \\multirow{4}{*}{cen2b}  '''+'& p0 &{0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2016['2bcen']['param'][0], params_2016['2bcen']['error'][0])+''' 
                     '''+'&       & p1 &{0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2016['2bcen']['param'][1], params_2016['2bcen']['error'][0])+''' 
                     '''+'&       & p2 &{0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2016['2bcen']['param'][2], params_2016['2bcen']['error'][0])+''' 
                     '''+'&       & p3 &{0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2016['2bcen']['param'][3], params_2016['2bcen']['error'][0])+'''  \\cline{2-5}
                     
                     & \\multirow{2}{*}{fwd0b} '''+'& p0 & {0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2016['0bfwd']['param'][0], params_2016['0bfwd']['error'][0])+'''
                     '''+'&       & p1 &  {0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2016['0bfwd']['param'][1], params_2016['0bfwd']['error'][1])+''' \\cline{2-5}
                     
                     & \\multirow{4}{*}{fwd1b} '''+'& p0 & {0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2016['1bfwd']['param'][0], params_2016['1bfwd']['error'][0])+''' 
                     '''+'&       & p1 & {0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2016['1bfwd']['param'][1], params_2016['1bfwd']['error'][1])+''' 
                     '''+'&       & p2 & {0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2016['1bfwd']['param'][2], params_2016['1bfwd']['error'][2])+'''
                     '''+'&       & p3 & {0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2016['1bfwd']['param'][3], params_2016['1bfwd']['error'][3])+'''  \\cline{2-5}
                     
                     & \\multirow{4}{*}{fwd2b} '''+'& p0 &{0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2016['2bfwd']['param'][0], params_2016['2bfwd']['error'][0])+''' 
                     '''+'&       & p1 &{0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2016['2bfwd']['param'][1], params_2016['2bfwd']['error'][1])+''' 
                     '''+'&       & p2 &{0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2016['2bfwd']['param'][2], params_2016['2bfwd']['error'][2])+''' 
                     '''+'&       & p3 &{0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2016['2bfwd']['param'][3], params_2016['2bfwd']['error'][3])+'''  
         
        \\hline
        \\end{tabular}
\\end{table}

\\begin{table}[htbp]
    \\centering
        \\label{tab:syst}
       \\caption{Values for parameters of the 2017 transfer function}
        \\begin{tabular}{|c||c||c||c||c|} \\hline 
            \\textbf{Year} & \\textbf{Region} & \\textbf{Parameter} & \\textbf{Value} & \\textbf{Uncertainty} \\\\ \\hline
            \\multirow{20}{*}{2017}
              
                     & \\multirow{2}{*}{cen0b}  '''+'& p0 & {0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2017['0bcen']['param'][0], params_2017['0bcen']['error'][0])+'''
                     '''+'&       & p1 &  {0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2017['0bcen']['param'][1], params_2017['0bcen']['error'][1])+''' \\cline{2-5}
                     
                     & \\multirow{4}{*}{cen1b}  '''+'& p0 & {0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2017['1bcen']['param'][0], params_2017['1bcen']['error'][0])+''' 
                     '''+'&       & p1 & {0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2017['1bcen']['param'][1], params_2017['1bcen']['error'][1])+''' 
                     '''+'&       & p2 & {0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2017['1bcen']['param'][2], params_2017['1bcen']['error'][2])+'''
                     '''+'&       & p3 & {0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2017['1bcen']['param'][3], params_2017['1bcen']['error'][3])+''' \\cline{2-5}
                     
                     & \\multirow{4}{*}{cen2b}  '''+'& p0 &{0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2017['2bcen']['param'][0], params_2017['2bcen']['error'][0])+''' 
                     '''+'&       & p1 &{0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2017['2bcen']['param'][1], params_2017['2bcen']['error'][0])+''' 
                     '''+'&       & p2 &{0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2017['2bcen']['param'][2], params_2017['2bcen']['error'][0])+''' 
                     '''+'&       & p3 &{0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2017['2bcen']['param'][3], params_2017['2bcen']['error'][0])+'''  \\cline{2-5}
                     
                     & \\multirow{2}{*}{fwd0b} '''+'& p0 & {0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2017['0bfwd']['param'][0], params_2017['0bfwd']['error'][0])+'''
                     '''+'&       & p1 &  {0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2017['0bfwd']['param'][1], params_2017['0bfwd']['error'][1])+''' \\cline{2-5}
                     
                     & \\multirow{4}{*}{fwd1b} '''+'& p0 & {0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2017['1bfwd']['param'][0], params_2017['1bfwd']['error'][0])+''' 
                     '''+'&       & p1 & {0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2017['1bfwd']['param'][1], params_2017['1bfwd']['error'][1])+''' 
                     '''+'&       & p2 & {0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2017['1bfwd']['param'][2], params_2017['1bfwd']['error'][2])+'''
                     '''+'&       & p3 & {0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2017['1bfwd']['param'][3], params_2017['1bfwd']['error'][3])+'''  \\cline{2-5}
                     
                     & \\multirow{4}{*}{fwd2b} '''+'& p0 &{0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2017['2bfwd']['param'][0], params_2017['2bfwd']['error'][0])+''' 
                     '''+'&       & p1 &{0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2017['2bfwd']['param'][1], params_2017['2bfwd']['error'][1])+''' 
                     '''+'&       & p2 &{0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2017['2bfwd']['param'][2], params_2017['2bfwd']['error'][2])+''' 
                     '''+'&       & p3 &{0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2017['2bfwd']['param'][3], params_2017['2bfwd']['error'][3])+'''  
         
        \\hline
        \\end{tabular}
\\end{table}


\\begin{table}[htbp]
    \\centering
        \\label{tab:syst}
       \\caption{Values for parameters of the 2018 transfer function}
        \\begin{tabular}{|c||c||c||c||c|} \\hline 
            \\textbf{Year} & \\textbf{Region} & \\textbf{Parameter} & \\textbf{Value} & \\textbf{Uncertainty} \\\\ \\hline
            \\multirow{20}{*}{2018}
              
                     & \\multirow{2}{*}{cen0b}  '''+'& p0 & {0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2018['0bcen']['param'][0], params_2018['0bcen']['error'][0])+'''
                     '''+'&       & p1 &  {0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2018['0bcen']['param'][1], params_2018['0bcen']['error'][1])+''' \\cline{2-5}
                     
                     & \\multirow{4}{*}{cen1b}  '''+'& p0 & {0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2018['1bcen']['param'][0], params_2018['1bcen']['error'][0])+''' 
                     '''+'&       & p1 & {0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2018['1bcen']['param'][1], params_2018['1bcen']['error'][1])+''' 
                     '''+'&       & p2 & {0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2018['1bcen']['param'][2], params_2018['1bcen']['error'][2])+'''
                     '''+'&       & p3 & {0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2018['1bcen']['param'][3], params_2018['1bcen']['error'][3])+''' \\cline{2-5}
                     
                     & \\multirow{4}{*}{cen2b}  '''+'& p0 &{0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2018['2bcen']['param'][0], params_2018['2bcen']['error'][0])+''' 
                     '''+'&       & p1 &{0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2018['2bcen']['param'][1], params_2018['2bcen']['error'][0])+''' 
                     '''+'&       & p2 &{0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2018['2bcen']['param'][2], params_2018['2bcen']['error'][0])+''' 
                     '''+'&       & p3 &{0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2018['2bcen']['param'][3], params_2018['2bcen']['error'][0])+'''  \\cline{2-5}
                     
                     & \\multirow{2}{*}{fwd0b} '''+'& p0 & {0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2018['0bfwd']['param'][0], params_2018['0bfwd']['error'][0])+'''
                     '''+'&       & p1 &  {0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2018['0bfwd']['param'][1], params_2018['0bfwd']['error'][1])+''' \\cline{2-5}
                     
                     & \\multirow{4}{*}{fwd1b} '''+'& p0 & {0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2018['1bfwd']['param'][0], params_2018['1bfwd']['error'][0])+''' 
                     '''+'&       & p1 & {0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2018['1bfwd']['param'][1], params_2018['1bfwd']['error'][1])+''' 
                     '''+'&       & p2 & {0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2018['1bfwd']['param'][2], params_2018['1bfwd']['error'][2])+'''
                     '''+'&       & p3 & {0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2018['1bfwd']['param'][3], params_2018['1bfwd']['error'][3])+'''  \\cline{2-5}
                     
                     & \\multirow{4}{*}{fwd2b} '''+'& p0 &{0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2018['2bfwd']['param'][0], params_2018['2bfwd']['error'][0])+''' 
                     '''+'&       & p1 &{0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2018['2bfwd']['param'][1], params_2018['2bfwd']['error'][1])+''' 
                     '''+'&       & p2 &{0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2018['2bfwd']['param'][2], params_2018['2bfwd']['error'][2])+''' 
                     '''+'&       & p3 &{0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2018['2bfwd']['param'][3], params_2018['2bfwd']['error'][3])+'''  
         
        \\hline
        \\end{tabular}
\\end{table}


'''
print(param_table)

In [ ]:
\\multirow{20}{*}{2018}
              
                     & \\multirow{2}{*}{cen0b}  '''+'& p0 & {0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2018['0bcen']['param'][0], params_2018['0bcen']['error'][0])+'''
                     '''+'&       & p1 &  {0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2018['0bcen']['param'][1], params_2018['0bcen']['error'][1])+''' \\cline{2-5}
                     
                     & \\multirow{4}{*}{cen1b}  '''+'& p0 & {0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2018['1bcen']['param'][0], params_2018['1bcen']['error'][0])+''' 
                     '''+'&       & p1 & {0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2018['1bcen']['param'][1], params_2018['1bcen']['error'][1])+''' 
                     '''+'&       & p2 & {0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2018['1bcen']['param'][2], params_2018['1bcen']['error'][2])+'''
                     '''+'&       & p3 & {0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2018['1bcen']['param'][3], params_2018['1bcen']['error'][3])+'''
                     '''+'&       & p4 & {0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2018['1bcen']['param'][4], params_2018['1bcen']['error'][4])+''' \\cline{2-5}
                     
                     & \\multirow{4}{*}{cen2b}  '''+'& p0 &{0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2018['2bcen']['param'][0], params_2018['2bcen']['error'][0])+''' 
                     '''+'&       & p1 &{0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2018['2bcen']['param'][1], params_2018['2bcen']['error'][1])+''' 
                     '''+'&       & p2 &{0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2018['2bcen']['param'][2], params_2018['2bcen']['error'][2])+''' 
                     '''+'&       & p3 &{0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2018['2bcen']['param'][3], params_2018['2bcen']['error'][3])+''' 
                     '''+'&       & p4 &{0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2018['2bcen']['param'][4], params_2018['2bcen']['error'][4])+'''  \\cline{2-5}
                     
                     & \\multirow{2}{*}{fwd0b} '''+'& p0 & {0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2018['0bfwd']['param'][0], params_2018['0bfwd']['error'][0])+'''
                     '''+'&       & p1 &  {0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2018['0bfwd']['param'][1], params_2018['0bfwd']['error'][1])+''' \\cline{2-5}
                     
                     & \\multirow{4}{*}{fwd1b} '''+'& p0 & {0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2018['1bfwd']['param'][0], params_2018['1bfwd']['error'][0])+''' 
                     '''+'&       & p1 & {0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2018['1bfwd']['param'][1], params_2018['1bfwd']['error'][1])+''' 
                     '''+'&       & p2 & {0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2018['1bfwd']['param'][2], params_2018['1bfwd']['error'][2])+'''
                     '''+'&       & p3 & {0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2018['1bfwd']['param'][3], params_2018['1bfwd']['error'][3])+'''
                     '''+'&       & p4 & {0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2018['1bfwd']['param'][4], params_2018['1bfwd']['error'][4])+'''  \\cline{2-5}
                     
                     & \\multirow{4}{*}{fwd2b} '''+'& p0 &{0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2018['2bfwd']['param'][0], params_2018['2bfwd']['error'][0])+''' 
                     '''+'&       & p1 &{0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2018['2bfwd']['param'][1], params_2018['2bfwd']['error'][1])+''' 
                     '''+'&       & p2 &{0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2018['2bfwd']['param'][2], params_2018['2bfwd']['error'][2])+''' 
                     '''+'&       & p3 &{0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2018['2bfwd']['param'][3], params_2018['2bfwd']['error'][3])+''' 
                     '''+'&       & p4 &{0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2018['2bfwd']['param'][4], params_2018['2bfwd']['error'][4])+'''  
         
        \\hline

In [ ]:
param_table_for_overleaf = '''
\\begin{frame}{Background Estimate: Transfer Function}
\\begin{table}[htbp]
    \\centering
        \\label{tab:syst}
       \\caption{Values for parameters of the 2016 transfer function}
        \\resizebox{0.5\\textwidth}{!}{%
        \\begin{tabular}{@{}|c||c||c||c||c|@{}}  \\hline 
            \\textbf{Year} & \\textbf{Region} & \\textbf{Parameter} & \\textbf{Value} & \\textbf{Uncertainty} \\\\ \\hline
            \\multirow{20}{*}{2016}
              
                     & \\multirow{2}{*}{cen0b}  '''+'& p0 & {0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2016['0bcen']['param'][0], params_2016['0bcen']['error'][0])+'''
                     '''+'&       & p1 &  {0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2016['0bcen']['param'][1], params_2016['0bcen']['error'][1])+''' \\cline{2-5}
                     
                     & \\multirow{4}{*}{cen1b}  '''+'& p0 & {0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2016['1bcen']['param'][0], params_2016['1bcen']['error'][0])+''' 
                     '''+'&       & p1 & {0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2016['1bcen']['param'][1], params_2016['1bcen']['error'][1])+''' 
                     '''+'&       & p2 & {0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2016['1bcen']['param'][2], params_2016['1bcen']['error'][2])+'''
                     '''+'&       & p3 & {0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2016['1bcen']['param'][3], params_2016['1bcen']['error'][3])+'''
                     '''+'&       & p4 & {0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2016['1bcen']['param'][4], params_2016['1bcen']['error'][4])+''' \\cline{2-5}
                     
                     & \\multirow{4}{*}{cen2b}  '''+'& p0 &{0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2016['2bcen']['param'][0], params_2016['2bcen']['error'][0])+''' 
                     '''+'&       & p1 &{0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2016['2bcen']['param'][1], params_2016['2bcen']['error'][1])+''' 
                     '''+'&       & p2 &{0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2016['2bcen']['param'][2], params_2016['2bcen']['error'][2])+''' 
                     '''+'&       & p3 &{0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2016['2bcen']['param'][3], params_2016['2bcen']['error'][3])+''' 
                     '''+'&       & p4 &{0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2016['2bcen']['param'][4], params_2016['2bcen']['error'][4])+'''  \\cline{2-5}
                     
                     & \\multirow{2}{*}{fwd0b} '''+'& p0 & {0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2016['0bfwd']['param'][0], params_2016['0bfwd']['error'][0])+'''
                     '''+'&       & p1 &  {0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2016['0bfwd']['param'][1], params_2016['0bfwd']['error'][1])+''' \\cline{2-5}
                     
                     & \\multirow{4}{*}{fwd1b} '''+'& p0 & {0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2016['1bfwd']['param'][0], params_2016['1bfwd']['error'][0])+''' 
                     '''+'&       & p1 & {0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2016['1bfwd']['param'][1], params_2016['1bfwd']['error'][1])+''' 
                     '''+'&       & p2 & {0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2016['1bfwd']['param'][2], params_2016['1bfwd']['error'][2])+'''
                     '''+'&       & p3 & {0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2016['1bfwd']['param'][3], params_2016['1bfwd']['error'][3])+'''
                     '''+'&       & p4 & {0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2016['1bfwd']['param'][4], params_2016['1bfwd']['error'][4])+'''  \\cline{2-5}
                     
                     & \\multirow{4}{*}{fwd2b} '''+'& p0 &{0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2016['2bfwd']['param'][0], params_2016['2bfwd']['error'][0])+''' 
                     '''+'&       & p1 &{0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2016['2bfwd']['param'][1], params_2016['2bfwd']['error'][1])+''' 
                     '''+'&       & p2 &{0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2016['2bfwd']['param'][2], params_2016['2bfwd']['error'][2])+''' 
                     '''+'&       & p3 &{0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2016['2bfwd']['param'][3], params_2016['2bfwd']['error'][3])+''' 
                     '''+'&       & p4 &{0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2016['2bfwd']['param'][4], params_2016['2bfwd']['error'][4])+'''  
         
        \\hline
        \\end{tabular}
        }
\\end{table}
\\end{frame}


\\begin{frame}{Background Estimate: Transfer Function}
\\begin{table}[htbp]
    \\centering
        \\label{tab:syst}
       \\caption{Values for parameters of the 2017 transfer function}
        \\resizebox{0.5\\textwidth}{!}{%
        \\begin{tabular}{@{}|c||c||c||c||c|@{}}  \\hline 
            \\textbf{Year} & \\textbf{Region} & \\textbf{Parameter} & \\textbf{Value} & \\textbf{Uncertainty} \\\\ \\hline
            \\multirow{20}{*}{2017}
              
                     & \\multirow{2}{*}{cen0b}  '''+'& p0 & {0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2017['0bcen']['param'][0], params_2017['0bcen']['error'][0])+'''
                     '''+'&       & p1 &  {0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2017['0bcen']['param'][1], params_2017['0bcen']['error'][1])+''' \\cline{2-5}
                     
                     & \\multirow{4}{*}{cen1b}  '''+'& p0 & {0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2017['1bcen']['param'][0], params_2017['1bcen']['error'][0])+''' 
                     '''+'&       & p1 & {0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2017['1bcen']['param'][1], params_2017['1bcen']['error'][1])+''' 
                     '''+'&       & p2 & {0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2017['1bcen']['param'][2], params_2017['1bcen']['error'][2])+'''
                     '''+'&       & p3 & {0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2017['1bcen']['param'][3], params_2017['1bcen']['error'][3])+'''
                     '''+'&       & p4 & {0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2017['1bcen']['param'][4], params_2017['1bcen']['error'][4])+''' \\cline{2-5}
                     
                     & \\multirow{4}{*}{cen2b}  '''+'& p0 &{0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2017['2bcen']['param'][0], params_2017['2bcen']['error'][0])+''' 
                     '''+'&       & p1 &{0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2017['2bcen']['param'][1], params_2017['2bcen']['error'][1])+''' 
                     '''+'&       & p2 &{0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2017['2bcen']['param'][2], params_2017['2bcen']['error'][2])+''' 
                     '''+'&       & p3 &{0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2017['2bcen']['param'][3], params_2017['2bcen']['error'][3])+''' 
                     '''+'&       & p4 &{0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2017['2bcen']['param'][4], params_2017['2bcen']['error'][4])+'''  \\cline{2-5}
                     
                     & \\multirow{2}{*}{fwd0b} '''+'& p0 & {0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2017['0bfwd']['param'][0], params_2017['0bfwd']['error'][0])+'''
                     '''+'&       & p1 &  {0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2017['0bfwd']['param'][1], params_2017['0bfwd']['error'][1])+''' \\cline{2-5}
                     
                     & \\multirow{4}{*}{fwd1b} '''+'& p0 & {0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2017['1bfwd']['param'][0], params_2017['1bfwd']['error'][0])+''' 
                     '''+'&       & p1 & {0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2017['1bfwd']['param'][1], params_2017['1bfwd']['error'][1])+''' 
                     '''+'&       & p2 & {0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2017['1bfwd']['param'][2], params_2017['1bfwd']['error'][2])+'''
                     '''+'&       & p3 & {0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2017['1bfwd']['param'][3], params_2017['1bfwd']['error'][3])+'''
                     '''+'&       & p4 & {0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2017['1bfwd']['param'][4], params_2017['1bfwd']['error'][4])+'''  \\cline{2-5}
                     
                     & \\multirow{4}{*}{fwd2b} '''+'& p0 &{0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2017['2bfwd']['param'][0], params_2017['2bfwd']['error'][0])+''' 
                     '''+'&       & p1 &{0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2017['2bfwd']['param'][1], params_2017['2bfwd']['error'][1])+''' 
                     '''+'&       & p2 &{0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2017['2bfwd']['param'][2], params_2017['2bfwd']['error'][2])+''' 
                     '''+'&       & p3 &{0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2017['2bfwd']['param'][3], params_2017['2bfwd']['error'][3])+''' 
                     '''+'&       & p4 &{0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2017['2bfwd']['param'][4], params_2017['2bfwd']['error'][4])+'''  
         
        \\hline
        \\end{tabular}
        }
\\end{table}
\\end{frame}

\\begin{frame}{Background Estimate: Transfer Function}
\\begin{table}[htbp]
    \\centering
        \\label{tab:syst}
       \\caption{Values for parameters of the 2018 transfer function}
        \\resizebox{0.5\\textwidth}{!}{%
        \\begin{tabular}{@{}|c||c||c||c||c|@{}}  \\hline 
            \\textbf{Year} & \\textbf{Region} & \\textbf{Parameter} & \\textbf{Value} & \\textbf{Uncertainty} \\\\ \\hline
            \\multirow{20}{*}{2018}
              
                     & \\multirow{2}{*}{cen0b}  '''+'& p0 & {0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2018['0bcen']['param'][0], params_2018['0bcen']['error'][0])+'''
                     '''+'&       & p1 &  {0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2018['0bcen']['param'][1], params_2018['0bcen']['error'][1])+''' \\cline{2-5}
                     
                     & \\multirow{4}{*}{cen1b}  '''+'& p0 & {0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2018['1bcen']['param'][0], params_2018['1bcen']['error'][0])+''' 
                     '''+'&       & p1 & {0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2018['1bcen']['param'][1], params_2018['1bcen']['error'][1])+''' 
                     '''+'&       & p2 & {0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2018['1bcen']['param'][2], params_2018['1bcen']['error'][2])+'''
                     '''+'&       & p3 & {0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2018['1bcen']['param'][3], params_2018['1bcen']['error'][3])+'''
                     '''+'&       & p4 & {0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2018['1bcen']['param'][4], params_2018['1bcen']['error'][4])+''' \\cline{2-5}
                     
                     & \\multirow{4}{*}{cen2b}  '''+'& p0 &{0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2018['2bcen']['param'][0], params_2018['2bcen']['error'][0])+''' 
                     '''+'&       & p1 &{0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2018['2bcen']['param'][1], params_2018['2bcen']['error'][1])+''' 
                     '''+'&       & p2 &{0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2018['2bcen']['param'][2], params_2018['2bcen']['error'][2])+''' 
                     '''+'&       & p3 &{0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2018['2bcen']['param'][3], params_2018['2bcen']['error'][3])+''' 
                     '''+'&       & p4 &{0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2018['2bcen']['param'][4], params_2018['2bcen']['error'][4])+'''  \\cline{2-5}
                     
                     & \\multirow{2}{*}{fwd0b} '''+'& p0 & {0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2018['0bfwd']['param'][0], params_2018['0bfwd']['error'][0])+'''
                     '''+'&       & p1 &  {0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2018['0bfwd']['param'][1], params_2018['0bfwd']['error'][1])+''' \\cline{2-5}
                     
                     & \\multirow{4}{*}{fwd1b} '''+'& p0 & {0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2018['1bfwd']['param'][0], params_2018['1bfwd']['error'][0])+''' 
                     '''+'&       & p1 & {0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2018['1bfwd']['param'][1], params_2018['1bfwd']['error'][1])+''' 
                     '''+'&       & p2 & {0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2018['1bfwd']['param'][2], params_2018['1bfwd']['error'][2])+'''
                     '''+'&       & p3 & {0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2018['1bfwd']['param'][3], params_2018['1bfwd']['error'][3])+'''
                     '''+'&       & p4 & {0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2018['1bfwd']['param'][4], params_2018['1bfwd']['error'][4])+'''  \\cline{2-5}
                     
                     & \\multirow{4}{*}{fwd2b} '''+'& p0 &{0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2018['2bfwd']['param'][0], params_2018['2bfwd']['error'][0])+''' 
                     '''+'&       & p1 &{0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2018['2bfwd']['param'][1], params_2018['2bfwd']['error'][1])+''' 
                     '''+'&       & p2 &{0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2018['2bfwd']['param'][2], params_2018['2bfwd']['error'][2])+''' 
                     '''+'&       & p3 &{0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2018['2bfwd']['param'][3], params_2018['2bfwd']['error'][3])+''' 
                     '''+'&       & p4 &{0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2018['2bfwd']['param'][4], params_2018['2bfwd']['error'][4])+'''  
         
        \\hline
        \\end{tabular}
        }
\\end{table}
\\end{frame}

'''
print(param_table_for_overleaf)